In [ ]:
import sys
import os
#os.environ['TF_CUDNN_DETERMINISTIC'] = 'true'
os.environ['TENSORPACK_FP16'] = 'true'
sys.path.append('/mask-rcnn-tensorflow/MaskRCNN')
import tensorflow as tf
import tqdm
import numpy as np
import tensorpack.utils.viz as tpviz
from tensorpack import *
from tensorpack.tfutils.common import get_tf_version_tuple
from model.generalized_rcnn import ResNetFPNModel
from config import finalize_configs, config as cfg
from eval import DetectionResult, predict_image, multithread_predict_dataflow, EvalCallback
from performance import ThroughputTracker, humanize_float
from data import get_eval_dataflow, get_train_dataflow, get_batch_train_dataflow
from train import do_visualize, do_evaluate, do_predict
from dataset import DetectionDataset
from viz import draw_annotation, draw_final_outputs, draw_predictions, draw_proposal_recall, gt_mask, apply_masks, draw_outputs
import matplotlib.pyplot as plt
from cv2 import cvtColor
from tensorpack.utils.palette import PALETTE_RGB
import cv2

In [ ]:
MODEL = ResNetFPNModel(True)
DetectionDataset()

In [ ]:
cfg.DATA.BASEDIR = '/data/coco/'
#cfg.DATA.BASEDIR = '/home/ec2-user/small_data'
cfg.BACKBONE.WEIGHTS = '/data/coco/pretrained-models/ImageNet-R50-AlignPadding.npz'
finalize_configs(is_training=False)

In [ ]:
trained_model = '/logs/model-11244.index'

In [ ]:
df = get_train_dataflow()

In [ ]:
df.reset_state()

In [ ]:
pred = OfflinePredictor(PredictConfig(
        model=MODEL,
        session_init=get_model_loader(trained_model),
        input_names=['images', 'orig_image_dims'],
        output_names=[
            'generate_{}_proposals_topk_per_image/boxes'.format('fpn' if cfg.MODE_FPN else 'rpn'),
            'generate_{}_proposals_topk_per_image/scores'.format('fpn' if cfg.MODE_FPN else 'rpn'),
            'fastrcnn_all_scores',
            'output/boxes',
            'output/scores',
            'output/labels',
            'output/masks'
        ]))

In [ ]:
an_image = next(df.get_data())
img, gt_boxes, gt_labels, gt_masks = an_image['images'], an_image['gt_boxes'], an_image['gt_labels'], an_image['gt_masks']
img = img[:,:,[2,1,0]]

In [ ]:
fig,ax = plt.subplots(figsize=(img.shape[1]//50, img.shape[0]//50))
ax.imshow(img.astype(int))

In [ ]:
gt_viz = draw_annotation(img, gt_boxes, gt_labels)
fig,ax = plt.subplots(figsize=(img.shape[1]//50, img.shape[0]//50))
ax.imshow(gt_viz.astype(int))

In [ ]:
rpn_boxes, rpn_scores, all_scores, final_boxes, final_scores, final_labels, masks = pred(np.expand_dims(img, axis=0),
                                                                                  np.expand_dims(np.array(img.shape), axis=0))

In [ ]:
final_viz = draw_outputs(img, final_boxes, final_scores, final_labels, threshold=0.8)
fig,ax = plt.subplots(figsize=(img.shape[1]//50, img.shape[0]//50))
ax.imshow(final_viz/255.)

In [ ]:
fig,ax = plt.subplots(figsize=(img.shape[1]//50, img.shape[0]//50))
ax.imshow(gt_mask(img, gt_masks)/255.)

In [ ]:
masked_image = apply_masks(img, final_boxes, masks, final_scores, score_threshold=.8, mask_threshold=0.5)
fig,ax = plt.subplots(figsize=(img.shape[1]//50, img.shape[0]//50))
ax.imshow(masked_image/255.)